In [ ]:
# string template

import os
from string import Template
import tempfile

from IPython.display import IFrame

content = Template('''
<!doctype html>
<head>
    <meta content="text/html;charset=utf-8" http-equiv="Content-Type">
    <meta content="utf-8" http-equiv="encoding">
</head>
<title>Basic Parallel Coordinates Example</title>

<link rel="stylesheet" type="text/css" href="../d3.parcoords.css">
<link rel="stylesheet" type="text/css" href="style.css">
<script src="lib/d3.js"></script>
<script src="lib/d3.svg.multibrush.js"></script>
<script src="lib/sylvester.js"></script>
<script src="lib/underscore.js"></script>
<script src="lib/slickgrid/jquery-1.7.min.js"></script>
<script src="lib/underscore.js"></script>
<script src="lib/underscore.math.js"></script>
<script src="../d3.parcoords.js"></script>

<body>
<p>Use this slider to change the curve smoothness. The current value is <strong id="smooth">0.0</strong><br/>
<input type="range" min="0" max="0.25" value="0.0" step="0.01" id="smoothness"/>
<p>Use this slider to change the bundling strength. The current value is <strong id="strength">0</strong><br/>
<input type="range" min="0" max="1" value="0" step="0.05" id="bundling"/>
<div id="bundleDimension">Bundle using clusters determined by: </div>
<div id="example" class="parcoords" style="width:900px;height:300px"></div>
</body>
<script>
// load csv file and create the chart

var data = $data //bind pandas dataframe to data var

// color scale for zscores
var colorscale = d3.scale.linear()
  .domain([0,1])
  .range(["blue", "red"])
  .interpolate(d3.interpolateLab);

var pc0 = d3.parcoords()("#example")
    .data(data)
    .bundlingStrength(0) // set bundling strength
    .smoothness(0)
    .bundleDimension("obj 0")
    .showControlPoints(false)
    .hideAxis(["name"])
    .composite("darken")
    .render()
    .brushMode("1D-axes")
    .reorderable()  // enable brushing
    .interactive();

pc0.svg.selectAll("text")
    .style("font", "10px");

change_color("obj 0");

// click label to activate coloring
pc0.svg.selectAll(".dimension")
    .on("click", change_color)
    .selectAll(".label")
    .style("font-size", "14px");

// smoothness
d3.select("#smoothness").on("change", function() {
    d3.select("#smooth").text(this.value);
    pc0.smoothness(this.value).render();
});

// bundling strength slider
d3.select("#bundling").on("change", function() {
    d3.select("#strength").text(this.value);
    pc0.bundlingStrength(this.value).render();
});

// update color
function change_color(dimension) {
  pc0.svg.selectAll(".dimension")
    .style("font-weight", "normal")
    .filter(function(d) { return d == dimension; })
    .style("font-weight", "bold")

  pc0.color(color(pc0.data(),dimension)).render()
}

// return color function based on plot and dimension
function color(col, dimension) {
  var z = normalize(_(col).pluck(dimension).map(parseFloat))
  return function(d) { return colorscale(z(d[dimension])) }
};

// color by zscore
function normalize(col) {
  var n = col.length,
  min = _(col).min()
  max = _(col).max()

  return function(d) {
    return (d-min)/(max-min);
  };
}

var select = d3.select("#bundleDimension").append("select").on("change", changeBundle);

var options = select.selectAll('option')
    .data(d3.keys(pc0.dimensions()));

options
    .enter()
    .append("option")
    .attr("value", function(d) {return d;})
    .text(function(d) {return d;});

function changeBundle() {
    pc0.bundleDimension(this.value);
}


</script>
''')


def data_to_string(data):
    '''format a pandas data frame in a string of dicts, with keys in the order
    they appear in the dataframe
    
    Parameters
    ----------
    data  : pd.Dataframe instance
    
    
    Returns
    -------
    str
    
    '''
    
    dict_data = data.to_dict(orient='records')
    ordered_keys = data.columns.values
    
    string_elements = []
    
    for entry in dict_data:
        elements = []
        for key in ordered_keys:
            value = entry[key]
            elements.append("'{}': {}".format(key, value))
        elements = ', '.join(elements)
        string_elements.append('{' + elements +'}')
    return "[" + ', '.join(string_elements) + "]"

class generate_html(object):
    '''
    
    Parameters
    ----------
    x : pandas dataframe
    y : dict of numpy arrays
    fn : str, optional
         file name, if not provided a temporary file is used
    
    '''
    
    # we need to keep a record of all the tempfiles, otherwise the
    # tempfiles are removed after the call, resulting in a 404 error
    tempfiles = []
    
    def __call__(self, df, fn=None):
        if fn is None:
            fh = tempfile.NamedTemporaryFile(suffix='.html', 
                        dir='.',
                        mode='w+t')
        else:
        
        
        data = data_to_string(df)

        formatted = content.substitute({'data':data})
        fh.write(formatted)
        fh.seek(0)
    
        if fn is None:
            self.tempfiles.append(fh)
        
        file_loc = './{}'
        file_path = tmpfile_loc.format(os.path.basename(fh.name))
        
        return IFrame(file_path, 1000,800)

# instantiate the class, and make it look as if it is a simple function
generate_html = plot_parallel()


        
    

# list of experiments
# each experiment has
# a key, outcome dict, and experiment dict